# Generate tools dynamically from `treelang` trees

This cookbook demonstrates how new tools can be created dynamically based on a `treelang` tree. 

Suppose you have a large set of very granular tools at your disposal, say for example, a full toolset of database functions. 

Some of the problems you may wish to solve may be easy to state at a high level but their implementation may be quite complex due to the low level tools.

In this cases, once you solve the problem using `treelang` once, you can create a high level tool specific for that high level problem that you can now re-use effectively without needing to infer the underlying tree again.

This supports the concept of dynamic MCP servers that can generate and serve tools autonomously.

In this simple scenario we simulate a gaming server supporting the following tools (`get_players`, `get_game_stats`, `average`) and try to create a single tool that specifically answers the question: "compute the average number of hours spent by <some_game> players on the <some_platform> platform".



## Imports

In [4]:
import asyncio

# mcp components for stdio communication
from mcp import ClientSession, StdioServerParameters, stdio_client

## Client

The interesting operations in the client code are the use of the `describe` function and the tool-generating `AST.tool` function.

The `describe` function uses an LLM to describe the given AST. More specifically, it adds a `name` and `description` to the "program" represented by this tree. 

These are used by the `AST.tool` function to create a named `Callable` (a function that you can pass to an MCP server, e.g. `mcp.add_tool(AST.tool(my_tree, session))`).

In [5]:
import os

from treelang.ai.arborist import EvalType, OpenAIArborist
from treelang.ai.provider import MCPToolProvider
from treelang.trees.tree import AST

USER_PROMPT = """
Can you tell me the average number of hours spent by {game} players from the {platform} community last month?
"""

async def main():
    # server parameters for running the gamestats server
    path = os.path.join(os.getcwd(), "gamestats.py")
    expected_stats = [1, 54, 233, 231, 64, 722, 43, 7, 23, 89, 23, 222]
    expected_result = sum(expected_stats) / len(expected_stats)

    server_params = StdioServerParameters(
        path=path, command="python", args=[path], env=None
    )

    # establish a communication channel with the server
    async with stdio_client(server_params) as (read, write):
        # create a client session for interaction
        async with ClientSession(read, write) as session:
            # initialize the session
            await session.initialize()
            # use the MCP tool provider to supply the tools
            provider = MCPToolProvider(session=session)

            arborist = OpenAIArborist(model="gpt-4o-2024-11-20", provider=provider)
            response = await arborist.eval(
                USER_PROMPT.format(game="Destiny 2", platform="Steam"), EvalType.TREE
            )
            # generate a name and description for the tree
            tree = await response.describe()
            print(
                f"creating new tool named {tree.name} with description {tree.description}"
            )
            tool = await AST.tool(tree, provider=provider)
            # run the new tool!
            result = await tool(game="Doom", feature="hours_played", platform="Sony")
            # check if the result is as expected
            assert (
                result == expected_result
            ), f"expected {expected_result}, got {result}"
            # success! we could add this tool to an MCP server for future use
            print(f"result: {result}")

## Main

In [6]:
loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-19' coro=<main() running at /tmp/ipykernel_32756/2633082749.py:11>>

creating new tool named calculateAverageGameHours with description Computes the average hours played for a specific game feature across players retrieved from a given platform.
result: 142.66666666666666
